In [1]:
import openai
import pandas as pd
import tqdm
import glob

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open("../apikey/apikey.txt", "r") as f:
    openai.api_key = f.readline().replace("\n", "")

In [3]:
def query_prompt(prompt, max_tokens=1000):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=0,
    )
    return response["choices"][0]["message"]["content"]

In [4]:
def get_df(fname):
    def f(categories):
        for cat in categories:
            if "science" in cat:
                return True
            if "physics" in cat:
                return True
            if "chemistry" in cat:
                return True
        return False
    
    def text_preprocess(text):
        return text.replace("===", "\n").replace("==", "\n")

    df = pd.read_parquet(fname)
    df_science = df[df["categories"].apply(f)]
    df_science["text"] = "title: " + df_science["title"] + "\n" + df_science["text"].apply(text_preprocess)
    return df_science.sample(len(df_science)//20)

In [5]:
files = glob.glob("../data/wikipedia/*.parquet")

In [6]:
import time

In [7]:
import pickle

In [8]:
import json

In [9]:
from datetime import datetime as dt

In [13]:
import traceback 
batch_size = 1

def make_prompt(series):
    prompt = f"""
# 依頼
You are a university professor and you are creating a test for your students.
Using the one given texts, create a question in which you must select one answer from five choices. Also, extract the part of the text that provides the basis for the answer.
The output should be an array in json format with the question text as 'prompt', the choices as 'A', 'B', 'C', 'D', and 'E', the answer choice as 'answer', and the rationale as 'basis'.
Also, please create one question per text. So the total number of problems created will be 1.

# text
## text 1 
{series['text']}


# attention
Please create one question per text. So the total number of problems created will be 1.
"""
    return prompt
texts = []
now_date = dt.now().strftime("%Y%m%d%H%M%S")

for f in files:
    df_science = get_df(f)
    
    for i in tqdm.tqdm(range(len(df_science)), desc=f):
        try:
            series = df_science.iloc[i]
            prompt = make_prompt(series)
            text = query_prompt(prompt)
            text = json.loads(text)
            if type(text) == list:
                text = text[0]
            text["wiki_id"] = series["id"]
            text["original_text"] = series["text"]
            texts.append(text)
        except Exception as e:
            print(e)
            traceback.print_exc()
            print(text)
    pd.DataFrame(texts).to_csv(f"output_gpt3.5_generate/{now_date}.csv")

../data/wikipedia/a.parquet:   7%|████▌                                                                 | 9/138 [00:40<09:17,  4.32s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_13518/2907329658.py", line 32, in <module>
    text = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, idx)
json.decoder.JSONDecodeError: Invalid \escape: line 3 column 23 (char 137)
../data/wikipedia/a.parquet:   7%|█████                                                                | 10/138 [00:52<14:21,  6.73s/it]

Invalid \escape: line 3 column 23 (char 137)
{
  "prompt": "What is the representation of the spacetime position paravector in the algebra of physical space?",
  "A": "x = x^0 + x^1 \mathbf{e}_1 + x^2 \mathbf{e}_2 + x^3 \mathbf{e}_3",
  "B": "x = \begin{pmatrix} x^0 + x^3 && x^1 - ix^2 \\\ x^1 + ix^2 && x^0-x^3\end{pmatrix}",
  "C": "x = \begin{pmatrix} E_3 & E_1 -i E_2 \\\ E_1 +i E_2 & -E_3 \end{pmatrix} \+ i \begin{pmatrix} B_3 & B_1 -i B_2 \\\ B_1 +i B_2 & -B_3 \end{pmatrix}",
  "D": "x = \partial \bar{A}",
  "E": "x = \frac{d x }{d \tau}",
  "answer": "A",
  "basis": "In APS, the spacetime position is represented as the paravector x = x^0 + x^1 \mathbf{e}_1 + x^2 \mathbf{e}_2 + x^3 \mathbf{e}_3, where the time is given by the scalar part , and e1, e2, e3 are the standard basis for position space."
}


../data/wikipedia/b.parquet:  40%|████████████████████████████▏                                         | 33/82 [02:40<03:56,  4.83s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_13518/2907329658.py", line 32, in <module>
    text = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, idx)
json.decoder.JSONDecodeError: Unterminated string starting at: line 9 column 12 (char 387)
../data/wikipedia/b.parquet:  41%|█████████████████████████████                                         | 34/82 [02:46<04:14,  5.31s/it]

Unterminated string starting at: line 9 column 12 (char 387)
{
  "prompt": "What is the main advantage of Bioglass compared to other synthetic bone grafting biomaterials?",
  "A": "It has anti-infective and angiogenic properties",
  "B": "It is degradable in body fluids",
  "C": "It can deliver ions beneficial for healing",
  "D": "It can form a coating of hydroxyapatite",
  "E": "It is biocompatible and bioactive",
  "answer": "A",
  "basis": "


../data/wikipedia/c.parquet:  36%|████████████████████████▌                                            | 46/129 [04:07<07:38,  5.53s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_13518/2907329658.py", line 32, in <module>
    text = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, idx)
json.decoder.JSONDecodeError: Expecting ',' delimiter: line 9 column 171 (char 426)
../data/wikipedia/c.parquet:  36%|█████████████████████████▏                                           | 47/129 [04:13<07:34,  5.55s/it]

...

../data/wikipedia/t.parquet:  31%|█████████████████████                                                | 65/213 [04:56<10:48,  4.38s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_13518/2907329658.py", line 31, in <module>
    text = query_prompt(prompt)
  File "/tmp/ipykernel_13518/2301454346.py", line 2, in query_prompt
    response = openai.ChatCompletion.create(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_resources/chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/openai/api_resources/abstract/engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 700, in _interpret_response
    self._interpret_response_l

This model's maximum context length is 16385 tokens. However, your messages resulted in 23661 tokens. Please reduce the length of the messages.
{'prompt': "What is Thomas Easton's latest nonfiction book?", 'A': 'Careers in Science', 'B': 'Taking Sides: Clashing Views on Controversial Issues in Science, Technology, and Society', 'C': 'Taking Sides: Clashing Views on Controversial Environmental Issues', 'D': 'Firefight', 'E': 'The Great Flying Saucer Conspiracy', 'answer': 'C', 'basis': 'His latest nonfiction books are Taking Sides: Clashing Views on Controversial Issues in Science, Technology, and Society (McGraw-Hill Dushkin, 8th ed., 2008), and Taking Sides: Clashing Views on Controversial Environmental Issues (McGraw- Hill Dushkin, 12th ed., 2007).', 'wiki_id': '9057828', 'original_text': 'title: Thomas Easton\nThomas A. Easton (born 17 July 1944) is a teacher and well-known science fiction critic and author. He retired as a professor from Thomas College of Maine in 2014 and now teac

../data/wikipedia/u.parquet:  22%|███████████████▊                                                       | 4/18 [00:17<01:01,  4.36s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_13518/2907329658.py", line 31, in <module>
    text = query_prompt(prompt)
  File "/tmp/ipykernel_13518/2301454346.py", line 2, in query_prompt
    response = openai.ChatCompletion.create(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_resources/chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/openai/api_resources/abstract/engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 700, in _interpret_response
    self._interpret_response_l

This model's maximum context length is 16385 tokens. However, your messages resulted in 19945 tokens. Please reduce the length of the messages.
{'prompt': 'What is unbounded nondeterminism in computer science?', 'A': 'A property of concurrency that guarantees that a request will eventually be serviced', 'B': 'A property of concurrency that guarantees that a request will be serviced immediately', 'C': 'A property of concurrency that guarantees that a request will never be serviced', 'D': 'A property of concurrency that guarantees that a request will be serviced randomly', 'E': 'A property of concurrency that guarantees that a request will be serviced in a bounded amount of time', 'answer': 'A', 'basis': 'In computer science, unbounded nondeterminism or unbounded indeterminacy is a property of concurrency by which the amount of delay in servicing a request can become unbounded as a result of arbitration of contention for shared resources while still guaranteeing that the request will eve

../data/wikipedia/z.parquet: 100%|████████████████████████████████████████████████████████████████████████| 8/8 [00:34<00:00,  4.35s/it]


In [ ]:
pd.DataFrame(texts)

In [41]:
df_science = get_df(files[0])
df_science[df_science["id"] == '29637793']

,id,title,text,categories


In [42]:
df_science

,id,title,text,categories
368626,1908395,Artificial brain,title: Artificial brain\nAn artificial brain (...,"[Computational neuroscience, Robotics, Emergin..."
34268,10410698,Abdul Amir al-Jamri,title: Abdul Amir al-Jamri\nSheikh Abdul Amir ...,"[1938 births, 2006 deaths, Deaths from kidney ..."
233077,1958222,Amyloid beta,title: Amyloid beta\nAmyloid beta (Aβ or Abeta...,"[Peptides, Molecular neuroscience, Alzheimer's..."
9870,3621668,A Woman of the Iron People,title: A Woman of the Iron People\nA Woman of ...,"[1991 American novels, 1991 science fiction no..."
139719,38366604,Albert Spaier,title: Albert Spaier\nAlbert Spaier (9 July 18...,"[1883 births, 1934 deaths, Writers from Iași, ..."
...,...,...,...,...
62685,4474244,Actuarial reserves,"title: Actuarial reserves\nIn insurance, an ac...","[Actuarial science, Capital requirement de:Dec..."
357456,4260564,Arrival II,title: Arrival II\nArrival ll (alternatively t...,"[1998 films, 1998 science fiction films, Ameri..."
391514,32894329,Astrobiophysics,title: Astrobiophysics\nAstrobiophysics is a f...,"[Astrophysics, Biophysics .]"
10000,2824171,A World of Difference (novel),title: A World of Difference (novel)\nA World ...,"[1990 American novels, Novels set during the C..."
